# Clustering Pruebas Images 

In this notebook, we will analyse the ["Pruebas" plant images](https://drive.google.com/file/d/1g2gHE5vp7-FfWnFTiO29vMLemUFyvCSM/view?usp=sharing). This data set contians 378 usable images and the zipped folder has been stored in the above link. All the images are 2832X 4240 color images.
Author: pc2846@g.rit.edu


## Cleaning the Data:

The folder contained some random images that had to be removed for this project. The images are captured in a time lapse, therefore we decided to group the images manually into 9 folders. We assumed that this manual process will help us clean the data and have a base for comparing the model's performance with the human's performance.

## Importing Essential Libraries

ScikitLearn and TensorFlow are the major APIs being used for modeling.

In [ ]:
import os
import numpy as np
import zipfile
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img

## Importing Dataset 
Downloading the zipped folder of dataset into the google colab from the drive. Later unzipping the folder to access the folders inside the main directory. We see 9 folders created after cleaning the dataset. 

In [ ]:
# Mounting drive to locate the dataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Unzip the archive
import zipfile
local_zip = '/content/drive/MyDrive/Quick Essential Docs./Quick_Projects/projects/Luciferase/train.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall()
zip_ref.close()

## Sanity Check Before Analysis
It is important to understand and check if all the images are accessible, we find 9 folders and some images stored in all of them. We count the number of images present in each of the folders so that we have a total number of files to work with.

In [ ]:
import os
base_dir = 'train'
lst_folders = os.listdir(base_dir)

# Sanity check
base_dir = 'train'
c1_dir = os.path.join(base_dir, 'c1')
c2_dir = os.path.join(base_dir, 'c2')
c3_dir = os.path.join(base_dir, 'c3')
c4_dir = os.path.join(base_dir, 'c4')
c5_dir = os.path.join(base_dir, 'c5')
c6_dir = os.path.join(base_dir, 'c6')
c7_dir = os.path.join(base_dir, 'c7')
c8_dir = os.path.join(base_dir, 'c8')
c9_dir = os.path.join(base_dir, 'c9')

# List of folders inside the train directory
lst_of_dirs = [c1_dir, c2_dir, c3_dir, c4_dir, c5_dir, c6_dir, c7_dir, c8_dir, c9_dir]

# Printing the number of files in each folder an the total number of images
sum_of_imgs = 0
for dir in lst_of_dirs:
  sum_of_imgs += len(os.listdir(dir))
  print('total training images in ' + dir + ':' , len(os.listdir(dir)))

print('\ntotal training images: ' + str(sum_of_imgs))
print()

# Printing few file names under each folder
print("File names in every folder: ")
for dir in lst_of_dirs:
  print(dir + ": " + str(os.listdir(dir)[:10]))

total training images in train/c1: 3
total training images in train/c2: 3
total training images in train/c3: 26
total training images in train/c4: 47
total training images in train/c5: 63
total training images in train/c6: 90
total training images in train/c7: 41
total training images in train/c8: 71
total training images in train/c9: 34

total training images: 378

File names in every folder: 
train/c1: ['DSC00001(1).JPG', 'DSC00005.JPG', 'DSC00004.JPG']
train/c2: ['DSC00001.JPG', 'DSC00002.JPG', 'DSC00003.JPG']
train/c3: ['DSC00022.JPG', 'DSC00021.JPG', 'DSC00014.JPG', 'DSC00013.JPG', 'DSC00017.JPG', 'DSC00029.JPG', 'DSC00011.JPG', 'DSC00020.JPG', 'DSC00030.JPG', 'DSC00023.JPG']
train/c4: ['DSC00032.JPG', 'DSC00063.JPG', 'DSC00068.JPG', 'DSC00050.JPG', 'DSC00058.JPG', 'DSC00060.JPG', 'DSC00045.JPG', 'DSC00062.JPG', 'DSC00061.JPG', 'DSC00059.JPG']
train/c5: ['DSC00107.JPG', 'DSC00114.JPG', 'DSC00110.JPG', 'DSC00098.JPG', 'DSC00115.JPG', 'DSC00125.JPG', 'DSC00140.JPG', 'DSC00104.JPG', 

## Analyzing the Images
Two images from each folder are displayed. We understand that a pattern exists where the brightness of the plants (bioluminescence) increases till a point and then starts to fall when the images are displayed in the captured order. 

In [ ]:
# Expectation: Since the images are time series, images from each foler must
# show us developement of plants in wells

# Utilities for displaying image to check the logic 
folder_file_dict = {}
for dir in lst_of_dirs:
  if dir not in folder_file_dict:
    name = dir[-2:] + '_files'
    folder_file_dict[name] = []

for k in folder_file_dict.keys():
  for dir in lst_of_dirs:
    if k[:2] == dir[-2:]:
      folder_file_dict[k].extend(os.listdir(dir))

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Sorted list of images
lst_of_cx_files = []
for v in folder_file_dict.values():
  lst_of_cx_files.append(sorted(v))

# List of 2 images 
pic_index = 2
display_lst = []
for cx_dir, cx_files in zip(lst_of_dirs, lst_of_cx_files):
  d = [os.path.join(cx_dir, fname) for fname in sorted(cx_files[pic_index-2:pic_index])]
  display_lst.append(d)

# List of images to be displayed
final_display = []
for d in display_lst:
  final_display += d


## Knowing the Size of Image
This step can help us understand our dataset better. We have to preprocess all the images into a size that is acceptable by the model that we will be using for feature extraction.

In [ ]:
# Choosing a random image
import secrets
rand_sample_path = secrets.choice(final_display)
sample_image  = load_img(rand_sample_path)
# Convert the image into its numpy array representation
sample_array = img_to_array(sample_image)
print(f"Random image has a shape of: {sample_array.shape}")

# Check if all the images are of the same shape
all_paths = []
for cx_dir, cx_files in zip(lst_of_dirs, lst_of_cx_files):
  l = [os.path.join(cx_dir, fname) for fname in cx_files[:]]
  all_paths.append(l)

flat_lst_paths = [item for l in all_paths for item in l]
rnd_img_shape = sample_array.shape
cnt = 0
tot_imgs = len(flat_lst_paths)
for path in flat_lst_paths:
  _image  = load_img(path)
  _array = img_to_array(_image)
  if rnd_img_shape == _array.shape:
    cnt += 1
if cnt == tot_imgs:
  print("\nAll images are of the same size")

## Data Preprocessing and Feature Extraction
The images in the raw form are of size 2832X4240 which is not acceptable by the VGG16 model. Therefore we resize the input images into 224X224 NumPy arrays. 

We will be using the VGG16 model as a feature extractor, this model takes in batches of images rather than a single one. Therefore we reshape the image as (1, 224, 224, 3). Below function takes one of our image and object of our instantiated model to preprocess the image and to return the features. The output layer of the VGG model is removed so that the new final layer is a fully-connected layer with 4,096 output nodes. This vector of 4,096 numbers is the feature vector that we will use to cluster the images.

In [ ]:
# Importing the model and methods for transfer learning
from keras.applications.vgg16 import VGG16 
from keras.models import Model
from keras.applications.vgg16 import preprocess_input 

# load the model first and pass as an argument
model = VGG16()
model = Model(inputs = model.inputs, outputs = model.layers[-2].output)

def extract_features(file, model):
    # load the image as a 224x224 array
    img = load_img(file, target_size=(224,224))
    # convert from image to numpy array
    img = img_to_array(img)
    # reshape the data for the model reshape(num_of_samples, dim 1, dim 2, channels)
    reshaped_img = img.reshape(1,224,224,3) 
    # prepare image for model
    imgx = preprocess_input(reshaped_img)
    # get the feature vector
    features = model.predict(imgx, use_multiprocessing=True)
    return features

# The extracted features are stored in the HashMap otherwise stored in the 
# pickle file
all_images = flat_lst_paths
data = {}
p = r"/content/plant_features.pkl"

# loop through each image in the dataset
for image in all_images:
    # try to extract the features and update the dictionary
    try:
        feat = extract_features(image, model)
        data[image] = feat
    # if something fails, save the extracted features as a pickle file (optional)
    except:
        with open(p,'wb') as file:
            pickle.dump(data, file)

1/1 [==============================] - 0s 34ms/step


In [ ]:
# List of the filenames
filenames = np.array(list(data.keys()))
# List of just the features
feat = np.array(list(data.values()))
# Reshaping the features
print(feat.shape)
feat = feat.reshape(-1,4096)
print(feat.shape)

(378, 1, 4096)
(378, 4096)


## Dimensionality Reduction with PCA
Inorder to address the Curse of Dimensionality we use PCA to reduce the number of features from 4096 to a smaller number. We statistically analyse the features to pick a small number for the dimensionality reduction. Typically, we want the explained variance to be between 95–99% from the below graph we statistically find out that **50** or **25** components will be the best for reduction.

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=50, svd_solver='full', random_state=22)
pca.fit(feat)
x = pca.transform(feat)
print("Features of " + str(feat.shape[0]) + " images before dimensionality reduction with PCA: " + str(feat.shape[1]))
print("Features of " + str(x.shape[0]) + " images before dimensionality reduction with PCA: " + str(x.shape[1]))

Features of 378 images before dimensionality reduction with PCA: 4096
Features of 378 images before dimensionality reduction with PCA: 50


## KMeans Clustering
## Checking the number of clusters with Elbow method.
### Looks like 3 clusters is the best fit for this scenario.

## KMeans training on 302 images

In [ ]:
x_train = x[:302]
x_validation = x[302:]

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3, random_state=22)
kmeans.fit(x_train)
labels = kmeans.labels_
print(str(len(np.unique(labels))) + " labels generated for " + str(len(labels)) + " images")
print("Unique ID's in generated labels: " + str(np.unique(labels)))

3 labels generated for 302 images
Unique ID's in generated labels: [0 1 2]


## Save K-Means model for CL and PCA as pickle file

In [ ]:
import pickle
pickle.dump(kmeans, open('kmeans_luciferase_cl.pkl', 'wb'))
pickle.dump(pca, open('pca_luciferase.pkl', 'wb'))